<a href="https://colab.research.google.com/github/yashthenuia/unsloth-finetune-ui/blob/main/colab/runner.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!git clone https://github.com/YOUR_USERNAME/unsloth-finetune-ui.git
%cd unsloth-finetune-ui/colab


In [ ]:
!pip install unsloth trl accelerate datasets bitsandbytes


In [ ]:
!python train.py \
  --model unsloth/Phi-3-mini-4k-instruct \
  --epochs 1 \
  --batch_size 2 \
  --lr 2e-4
